Data cleaning for Mistral results

In [1]:
with open('outputs/mistral_few_ans.json', 'r') as f:
    mistral_few_ans = json.load(f)

with open('outputs/mistralSE_few_ans.json', 'r') as f:
    mistral_SE_few_ans = json.load(f)

with open('outputs/answers_mistral_zero.json', 'r') as f:
    mistral_zero_ans = json.load(f)

with open('outputs/mistral_few_strict.json', 'r') as f:
    mistral_few_strict = json.load(f)

with open('outputs/mistralSE_few_strict.json', 'r') as f:
    mistral_SE_few_strict = json.load(f)

FileNotFoundError: [Errno 2] No such file or directory: 'outputs/mistral_few_ans.json'

In [ ]:
def clean_mistral(data):
    final_ans_dict = {}
    pattern1 = r'\{.*?\}'

    for key, value in data.items():
        answer_list = []
        # print(key)
        for item in value:
            item = item.replace('\n', '').replace('%', '')
            # Add a closing parenthesis if missing
            if item.count('{') > item.count('}'):
                item += "}"
            # Correct wrong number format
            item = item.replace('.}', '.0}').replace(':}', ':0}')
            # Only match with the parenthesis and not any additional content
            match = re.search(pattern1, item)
            matched = match.group(0)
            final_ans = eval(matched)  

            answer_list.append(final_ans)
            # Check if each dict has right amount of answers
            # Valid number of answers
        valid_ans = anes[anes[key] > 0]
        number_answers = len(valid_ans[key].unique())
        # LLM number of answers
        ans_dict = [i for i in answer_list if len(i) == number_answers]

        final_ans_dict[key] = ans_dict
    return final_ans_dict
        
mistral_few_ans = clean_mistral(mistral_few_ans)
mistral_SE_few_ans = clean_mistral(mistral_SE_few_ans)
mistral_zero_ans = clean_mistral(mistral_zero_ans)
mistral_few_strict = clean_mistral(mistral_few_strict)
mistral_SE_few_strict = clean_mistral(mistral_SE_few_strict)

In [ ]:
def correct_check(answer_object):
    for key, answers in answer_object.items():
        
        #check if answers add up to 100%
        valid_answer = []
        for i in answers:
            if sum(i.values()) == 100:
                valid_answer.append(i)

            # If not normalize for 100
            else:
                # print(f"Sum is not equal to 100 in answer: {key}, {sum(i.values())}")
                sum_val = sum(i.values())
               
                for quest, ele in i.items():
                    norm = round(ele*100/sum_val, 1)
                    i[quest] = norm
                    valid_answer.append(i)

        if valid_answer:  # If there are valid answers
            array_data = np.array([list(dic.values()) for dic in valid_answer])
            averages = np.mean(array_data, axis=0)
            averages = np.round(averages, 1)
            av_dict = dict(enumerate(averages.flatten(), 1))
            answers.append(av_dict)
        else:
            print("No valid answers found.")
correct_check(mistral_few_ans)